In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [3]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [22]:
output = t2("a b c d e f g h i g k l.",truncation=True,padding='max_length',max_length=20)

In [3]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'test'

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
path = manager.path + 'MIND/MINDdemo_train/'
a = MIND(manager, path + 'news.tsv', path + 'behaviors.tsv')

[2021-09-26 15:21:51,826] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors..pkl
[2021-09-26 15:21:51,842] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-09-26 15:21:52,728] INFO (utils.utils) deduplicating...


In [2]:
subwords = pickle.load(open('data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))

In [3]:
subwords['subwords_all']

array([list([[0, 0], [1, 1], [0, 0], [1, 2], [0, 0], [1, 3], [0, 0], [1, 4], [0, 0], [1, 5], [0, 0], [1, 6], [0, 0], [1, 7], [0, 0], [1, 8], [0, 0], [1, 9], [0, 0], [1, 10], [0, 0], [1, 11], [0, 0], [1, 12], [0, 0], [1, 13], [0, 0], [1, 14], [0, 0], [1, 15], [0, 0], [1, 16], [0, 0], [1, 17], [0, 0], [1, 18], [0, 0], [1, 19], [0, 0], [1, 20], [0, 0], [1, 21], [0, 0], [1, 22], [0, 0], [1, 23], [0, 0], [1, 24], [0, 0], [1, 25], [0, 0], [1, 26], [0, 0], [1, 27], [0, 0], [1, 28], [0, 0], [1, 29], [0, 0], [1, 30], [0, 0], [1, 31], [0, 0], [1, 32], [0, 0], [1, 33], [0, 0], [1, 34], [0, 0], [1, 35], [0, 0], [1, 36], [0, 0], [1, 37], [0, 0], [1, 38], [0, 0], [1, 39], [0, 0], [1, 40], [0, 0], [1, 41], [0, 0], [1, 42], [0, 0], [1, 43], [0, 0], [1, 44], [0, 0], [1, 45], [0, 0], [1, 46], [0, 0], [1, 47], [0, 0], [1, 48], [0, 0], [1, 49], [0, 0], [1, 50], [0, 0], [1, 51], [0, 0], [1, 52], [0, 0], [1, 53], [0, 0], [1, 54], [0, 0], [1, 55], [0, 0], [1, 56], [0, 0], [1, 57], [0, 0], [1, 58], [0, 0], [1

In [14]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)

[CLS] 1 [0 0] 1
former 1 [1 1] 1
us 1 [2 2] 1
senator 1 [3 3] 1
kay 1 [4 4] 1
ha 1 [5 5] 1
##gan 1 [5 6] 1
dead 1 [6 7] 1
at 1 [7 8] 1
66 1 [8 9] 1
former 1 [ 9 10] 0
u 1 [10 11] 1
. 1 [11 12] 0
s 1 [12 13] 1
. 1 [13 14] 0
sen 1 [14 15] 1
. 1 [15 16] 0
kay 1 [16 17] 0
ha 1 [17 18] 0
##gan 1 [17 19] 0
, 1 [18 20] 0
a 1 [19 21] 1
one 1 [20 22] 1
- 1 [21 23] 0
time 1 [22 24] 1
capitol 1 [23 25] 1
hill 1 [24 26] 1
intern 1 [25 27] 1
who 1 [26 28] 1
went 1 [27 29] 1
on 1 [28 30] 1
to 1 [29 31] 1
become 1 [30 32] 1
north 1 [31 33] 1
carolina 1 [32 34] 1
##s 1 [32 35] 1
first 1 [33 36] 1
democratic 1 [34 37] 1
female 1 [35 38] 1
senator 1 [36 39] 0
, 1 [37 40] 0
died 1 [38 41] 1
monday 1 [39 42] 1
at 1 [40 43] 0
her 1 [41 44] 1
greensboro 1 [42 45] 1
home 1 [43 46] 1
. 1 [44 47] 0
she 1 [45 48] 1
was 1 [46 49] 1
66 1 [47 50] 0
. 1 [48 51] 0
news 1 [49 52] 1
##pol 1 [49 53] 1
##itic 1 [49 54] 1
##s 1 [49 55] 0
[SEP] 1 [50 56] 1
[PAD] 0 [0 0] 0
[PAD] 0 [0 0] 0
[PAD] 0 [0 0] 0
[PAD] 0 [0 0] 0
[P

In [3]:
x = a[1]
x['user_index']

2

In [ ]:
cdd = x['cdd_encoded_index'][0]

In [ ]:
subword = x['cdd_subword_index'][0]

In [ ]:
manager.convert_tokens_to_words(t.convert_ids_to_tokens(cdd))

In [ ]:
subword

In [ ]:
t.convert_ids_to_tokens(cdd)

In [ ]:
t.convert_ids_to_tokens([   1, 7565,   16, 4690,    0,    0,    0,    0])

In [ ]:
tokens = t.tokenize("[CLS] I love you embeddings.")
tokens

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

In [ ]:
t.convert_tokens_to_ids([i for i in r"[.&*()+=/\<>,!?;:~`@#$%^]"])